In [1]:
from mapping_reef_classes import get_coordinates
import os
import re
import pandas as pd
import numpy as np

In [2]:
def list_files(filepath, filetype):
   paths = []
   for root, dirs, files in os.walk(filepath):
      for file in files:
         if file.lower().endswith(filetype.lower()):
            paths.append(os.path.join(root, file))
   return(paths)

def list_collects(filepath):
   paths = []
   for root, dirs, files in os.walk(filepath):
      for dir in dirs:
        paths.append(os.path.join(root, dir))
   collects = [re.findall('.*\d+_\d+_\w+_\w{4}\Z', i) for i in paths]
   collects = list(set([item for sublist in collects for item in sublist]))
   collects.sort()
   return(collects)

In [3]:
tif_files = list_files(r"C:\Users\ageglio\OneDrive - DOI\Documents - Reef Mapping\Data Release", 'tif')
# further sorting of file search
tif_files = [re.findall('.*Classified.*', i) for i in tif_files]
tif_files = list(set([item for sublist in tif_files for item in sublist]))
tif_files.sort()

In [4]:
df_tiffs = get_coordinates().return_min_max_tif_df(tif_files)
# df_tiffs.to_csv("reef_coordinates.csv")
df_tiffs

,filename,min_lat,min_lon,max_lat,max_lon,avr_lat,avr_lon
0,BayHarbor_Classified2.tif,45.37037607752312,-85.00011331072805,45.378666926461676,-84.98020254687123,45.374522,-84.990158
1,ElkRapids_Classified2.tif,44.895292239871765,-85.42900281296895,44.916074870460776,-85.40411128722691,44.905684,-85.416557
2,FishermensIsland_Classified.tif,45.286371114752676,-85.37227014158857,45.302994277286366,-85.3451612323232,45.294683,-85.358716
3,IngallsPoint_Classified.tif,45.0677311176502,-85.57785037097511,45.07712022880579,-85.56420101824261,45.072426,-85.571026
4,MissionPeninsula_Classified_test.tif,44.984332675468785,-85.5050507857167,45.03571383282881,-85.46958703081921,45.010023,-85.487319
5,TanneryCreek_Classified.tif,45.39100464894159,-84.9329994177815,45.409447517286985,-84.9175919590057,45.400226,-84.925296
6,TraverseShoal_Classified.tif,45.12513982247555,-85.44574920242687,45.15228046679968,-85.42085308539035,45.138710,-85.433301


In [5]:
collects2019 = list_collects(r"W:\2019") + list_collects(r"M:\2019")
# collects2020 = list_collects(r"W:\2020")
# collects2021 = list_collects(r"Z:\NewestCollects\2021") + list_collects(r"W:\2021")
# collects2022 = list_collects(r"Z:\NewestCollects\2022")
# collects2023 = list_collects(r"Z:\NewestCollects\2023") + list_collects(r"X:\AUV Collects\2023")
# all_collects = collects2019 + collects2020 + collects2021 + collects2022 + collects2023
# df_collects_latlon = get_coordinates().return_MissionLog_min_max_df(all_collects)
# df_collects_latlon.to_csv("collects_lat_lon.csv")

# df = pd.read_csv("collects_lat_lon.csv", index_col=0)

In [10]:
BayHarbor = df_tiffs.loc[0]
ElkRapids = df_tiffs.loc[1]
FishermansIsland = df_tiffs.loc[2]
IngallsPoint = df_tiffs.loc[3]
MissionPeninsula = df_tiffs.loc[4]
TanneryCreek = df_tiffs.loc[5]
TraverseShoal = df_tiffs.loc[6]

In [ ]:
def reef_overlap(reef, df):
    new_df = df[(reef.avr_lat > df.min_lat) & (reef.avr_lat < df.max_lat) & (reef.avr_lon > df.min_lon) & (reef.avr_lon < df.max_lon)]
    new_df['Reef_overlap'] = [f"{reef.filename}"]*len(new_df)
    return new_df

BayHarbor_df = reef_overlap(BayHarbor, df)
ElkRapids_df = reef_overlap(ElkRapids, df)
FishermansIsland_df = reef_overlap(FishermansIsland, df)
IngallsPoint_df = reef_overlap(IngallsPoint, df)
MissionPeninsula_df = reef_overlap(MissionPeninsula, df)
TanneryCreek_df = reef_overlap(TanneryCreek, df)
TraverseShoal_df = reef_overlap(TraverseShoal, df)

dfs = [BayHarbor_df, ElkRapids_df, FishermansIsland_df, IngallsPoint_df, MissionPeninsula_df, TanneryCreek_df, TraverseShoal_df]
reef_overlap = pd.concat(dfs)


In [21]:
reef_overlap.sort_values(by=["Reef_overlap", "date"])
reef_overlap.to_csv("Reef_tif_collects_overlap.csv")